In [ ]:
'''
RGH 80
0105860101
'''

1 define path

In [76]:
import os
from glob import glob

In [8]:
homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/reduction'
obsid = '0105860101'
os.chdir(homepath)

2 Initialize steps

In [5]:
f = open('cifbuild.sh', 'w')
f.write(
f'''
export SAS_ODF={homepath}
cifbuild 2>&1 | tee cifbuild.log

'''
)
f.close()




In [8]:
f = open('odfingest.sh', 'w')
f.write(f'''
export SAS_CCF={homepath}/ccf.cif
export SAS_ODF={homepath}
odfingest odfdir=$SAS_ODF outdir=$SAS_ODF 2>&1 | tee odfingest.log
''')
f.close()

3 Set SAS environment

In [77]:
homepath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/reduction/SDSSTG3460'

f = open('set_sas.txt', 'w')
sasfile= glob('*SUM.SAS')[0] 
f.write(f'''
export SAS_CCF={homepath}/ccf.cif
export SAS_ODF={homepath}/{sasfile}
sasversion
''')
f.close()

4 emproc

In [14]:
f = open('emproc.sh', 'w')
f.write(
'''
emproc 2>&1 |tee emproc.log
mv *EMOS1_S001_ImagingEvts.ds mos1-S001.evt
mv *EMOS2_S002_ImagingEvts.ds mos2-S002.evt
'''
)
f.close()

5 epproc

In [15]:
f = open('epproc.sh', 'w')
f.write(
'''
epproc 2>&1 |tee epproc.log
mv *ImagingEvts.ds pn-S003.evt
# epproc withoutoftime=true 2>&1 |tee epproc_oot.log
# mv *EPN_S003_ImagingEvts.ds pn-oot-S003.evt
'''
)
f.close()

6 emanom: diagnose abnormal bright chips in MOS
NO NEED! the esas has run the *-anom.log!

In [18]:
f = open('emanom.sh', 'w')
f.write(
'''
emanom eventfile=mos1-S001.evt keepcorner=no 2>&1 | tee emanom_mos1.log
emanom eventfile=mos2-S002.evt keepcorner=no 2>&1 | tee emanom_mos2.log
'''
)
f.close()

# mos2 chip5 is I, but not serious by visually inspect, not excluded. 
# This observation is indeed contaminated by SP. (40% exp time is flared)


In [81]:
f = open('set_chips_on.txt', 'w')
f.write(
'''
export M1ON="T T F T T F T"
export M2ON="T T T T F T T"
export PNON="T T T T"
'''
)
f.close()

In [21]:
# visual inspect
f = open('chips_visual_inspect.sh', 'w')
f.write(
'''
evselect table=mos1-S001.evt withimageset=yes imageset=mos1S001-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&&(PATTERN<=12)&&((FLAG & 0x766aa000)==0)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
evselect table=mos2-S002.evt withimageset=yes imageset=mos2S002-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&& (PATTERN<=12)&&((FLAG & 0x766aa000)==0)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
evselect table=pn-S003.evt withimageset=yes imageset=pnS003-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&& (PATTERN <= 4)&&(#XMMEA_EP)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
evselect table=pn-oot-S003.evt withimageset=yes imageset=pnS003-oot-diag-det-unfilt.fits filtertype=expression expression="(PI in [300:1000])&& (PATTERN <= 4)&&(#XMMEA_EP)" ignorelegallimits=yes imagebinning=imageSize xcolumn=DETX ximagesize=780 ximagemax=19500 ximagemin=-19499 ycolumn=DETY yimagesize=780 yimagemax=19500 yimagemin=-19499
'''
)
f.close()


7 SP flare filtering

In [24]:
f = open('sp_flare.sh', 'w')
f.write(
'''
espfilt eventfile=mos1-S001.evt elow=2500 ehigh=8500 withsmoothing=yes smooth=51 rangescale=6.0 allowsigma=3.0 method=histogram keepinterfiles=false 2>&1 | tee flare_filter_mos1-S001.log
espfilt eventfile=mos2-S002.evt elow=2500 ehigh=8500 withsmoothing=yes smooth=51 rangescale=6.0 allowsigma=3.0 method=histogram keepinterfiles=false 2>&1 | tee flare_filter_mos2-S002.log
espfilt eventfile=pn-S003.evt elow=2500 ehigh=8500 withsmoothing=yes smooth=51 rangescale=15.0 allowsigma=3.0 method=histogram withoot=Y ootfile=pn-oot-S003.evt keepinterfiles=false 2>&1 | tee flare_filter_pn-S003.log
'''
)

f.close()
# except for P-allevc.fits and P-gti.fits are kept, others are moved to DIAGNOISE dir after checking


8 check sp flare filtering diagnostic files
Files: P-hist.qdp
Conclusion: 
not very flared


9 Remove the point sources

In [87]:
f = open('cheese.sh', 'w')
f.write(
'''
elo=350 
ehi=8000
cheese mos1file='mos1S001-allevc.fits' mos2file='mos2S002-allevc.fits' pnfile='pnS003-allevc.fits' pnootfile='pnS003-allevcoot.fits' elowlist=$elo ehighlist=$ehi scale=0.5 mlmin=1 dist=1. ratetotal=3e-6 keepinterfiles=no 2>&1 | tee cheese_rate0001_ml01_$elo-$ehi_dist1.log
'''
)
f.close()

10 visual inspect and alter emllist.fits 
blink P-fovimt.fits and P-cheeset.fits

In [21]:
# alter emllist.fits to emllist.reg file
from astropy.io import fits
import numpy as np

print(os.getcwd())
filename = f'emllist.fits'

f = fits.open(filename)
dat = f[1].data
print(len(dat))
f.close()

# load the regions of complete pointsources
newf = open(f'emllist.reg', 'w')
newf.write('fk5\n')

for i in range(len(dat)//4):
    newf.write(f'''circle({dat[4*i+1]['RA']}, {dat[4*i+1]['DEC']}, 10")\n''')
    
newf.close()


/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/reduction
448


In [19]:
10//2

5

In [83]:
from astropy.io import fits
import numpy as np

'''
Note: emllist_complete.fits only has reasonable ra dec, x_ima, y_ima, the rest values are wrong. 
'''

# 
filename = f'{homepath}/emllist.fits'
os.system(f'cp {filename} {filename.split(".")[0]}_complete.fits')
newfilename = f'{filename.split(".")[0]}_complete.fits'

# 
f = fits.open(filename)
dat = f[1].data
f.close()
dat['FLAG'] = 0


# load the styles of the recarray data 
sample_dype = dat.dtype
sample = dat[0]

# load the regions of complete pointsources
srcname = f'{homepath}/all_sources.reg'
regfile = open(srcname)
lines = regfile.readlines()
regfile.close()

# 
num = len(lines)-3
print(num)
dat = dat[:num]

# 
for i, f in enumerate(lines[3:]):
    ra = f.split('(')[-1].split(',')[0]
    dec = f.split(',')[1]
    dat[i]['RA'] = ra
    dat[i]['DEC'] = dec
    dat[i]['ML_ID_SRC'] = int(i+1)
    dat[i]['DIST_NN'] = 1000
    dat[i]['DET_ML'] = 20
    dat[i]['BOX_ID_SRC'] = int(i+1)
    dat[i]['ID_INST'] = 0
    dat[i]['ID_BAND'] = 0
    dat[i]['ID_CLUSTER'] = int(i+1)


newf = fits.open(f'{filename.split(".")[0]}_complete.fits', mode='update')
newf[1].data = dat
newf.flush()
newf.close()


32


In [84]:
# alter emllist.fits to emllist.reg file
from astropy.io import fits
import numpy as np

print(os.getcwd())
filename = f'emllist_complete.fits'

f = fits.open(filename)
dat = f[1].data
print(len(dat))
f.close()

# load the regions of complete pointsources
newf = open(f'emllist_complete.reg', 'w')
newf.write('fk5\n')

for i in range(len(dat)):
    newf.write(f'''circle({dat[i]['RA']}, {dat[i]['DEC']}, 10")\n''')
    
newf.close()


/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/reduction/SDSSTG3460
32


11 region & makemask

In [85]:
## edit region files

f = open(f'makemask.sh', 'w')

f.write(
'''
export M1=mos1S001
export M2=mos2S002
export PN=pnS003

for name in $M1 $M2 $PN
do
mkdir cheese_old
mv ${name}-cheeset.fits cheese_old
mv ${name}-bkgregtdet.fits cheese_old
mv ${name}-bkgregtsky.fits cheese_old
region eventset=${name}-allevc.fits operationstyle=global srclisttab=emllist_complete.fits:SRCLIST expression="(ID_INST == 0)&&(DET_ML >= 1)" bkgregionset=${name}-bkgregtdet.fits energyfraction=0.4 radiusstyle=contour outunit=detxy verbosity=4 2>&1 | tee region-det-${name}.log
region eventset=${name}-allevc.fits operationstyle=global srclisttab=emllist_complete.fits:SRCLIST expression="(ID_INST == 0)&&(DET_ML >= 1)" bkgregionset=${name}-bkgregtsky.fits energyfraction=0.4 radiusstyle=contour outunit=xy verbosity=4 2>&1 | tee region-sky-${name}.log

makemask imagefile=${name}-fovimt.fits maskfile=${name}-fovimtmask.fits regionfile=${name}-bkgregtsky.fits cheesefile=${name}-cheeset.fits 2>&1 | tee makemask-${name}.log
done

'''
)

f.close()

12 create the extraction region
bkg src


In [37]:
from astropy.coordinates import SkyCoord
import astropy.units as u

def deg2sex(ra_deg, dec_deg):
    # Create a SkyCoord object with the specified units
    coords = SkyCoord(ra=ra_deg * u.degree, dec=dec_deg * u.degree)
    # Extract the converted sexagesimal values
    ra_sexagesimal = coords.to_string(style='hmsdms', precision=2)
    return f"{ra_sexagesimal.split(' ')[0]}", f"{ra_sexagesimal.split(' ')[1]}"



In [78]:
os.chdir(f'/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/reduction/SDSSTG3460')

center_coor_deg = (170.735,34.1115)

# bkg 12.1242',14.799' 
# reg10 6.48967',8.0376' 

ra, dec = deg2sex(center_coor_deg[0], center_coor_deg[1])

f = open('deg2det.sh', 'w')

f.write(
f'''
export M1=mos1S001
export M2=mos2S002
export PN=pnS003

esky2det datastyle=user ra={ra} dec={dec} outunit=det calinfoset='mos1S001-allevc.fits' 2>&1 | tee deg2det-$M1.log
esky2det datastyle=user ra={ra} dec={dec} outunit=det calinfoset='mos2S002-allevc.fits' 2>&1 | tee deg2det-$M2.log
esky2det datastyle=user ra={ra} dec={dec} outunit=det calinfoset='pnS003-allevc.fits' 2>&1 | tee deg2det-$PN.log
''')

f.close()





In [97]:
''' RGH 80
   307.6     -183.9
   15.1       90.4
   132.2     -235.2

'''

detx_dict = {'mos1S001':307.6, 'mos2S002':15.1, 'pnS003':132.2}
dety_dict = {'mos1S001':-183.9, 'mos2S002':90.4, 'pnS003':-235.2}

In [92]:
print(os.getcwd())
'''ID3460
294.8     -203.6
34.7       77.5
112.5     -222.4


'''

detx_dict = {'mos1S001':294.8 , 'mos2S002':34.7, 'pnS003':112.5}
dety_dict = {'mos1S001':-203.6, 'mos2S002':77.5, 'pnS003':-222.4}

/Users/eusracenorth/Documents/work/XGAP-ABUN/data/ID3460/reduction/SDSSTG3460


In [98]:
os.chdir('/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/reduction')

reg_radius = [12.1242, 14.799] # arcmin


for name in ['mos1S001', 'mos2S002', 'pnS003']:
    f = open(f'bkg-{name}.reg', 'w')

    f.write(
    f'''&&((DETX,DETY) IN annulus({detx_dict[name]},{dety_dict[name]},{reg_radius[0]*60/0.05},{reg_radius[1]*60/0.05}))''')

    f.close()


13 mosspectra mosback

In [99]:
print(os.getcwd())
f = open('mosspectra_bkg.sh', 'w')
f.write(
'''
export ELO=700
export EHI=1200
export M1=mos1S001
export M2=mos2S002
export regname=bkg
source set_chips_on.txt

mosspectra eventfile=${M1}-allevc.fits keepinterfiles=yes withregion=yes regionfile=${regname}-${M1}.reg pattern=12 withsrcrem=yes maskdet=${M1}-${regname}regtdet.fits masksky=${M1}-${regname}regtsky.fits elow=${ELO} ehigh=${EHI} ccds="${M1ON}" -V=7 2>&1 |tee mosspectra_${M1}.log
mosback inspecfile=${M1}-fovt.pi elow=${ELO} ehigh=${EHI} ccds="${M1ON}" 2>&1 |tee mosback_${M1}.log
mosspectra eventfile=${M2}-allevc.fits keepinterfiles=yes withregion=yes regionfile=${regname}-${M2}.reg pattern=12 withsrcrem=yes maskdet=${M2}-${regname}regtdet.fits masksky=${M2}-${regname}regtsky.fits elow=${ELO} ehigh=${EHI} ccds="${M2ON}" -V=7 2>&1 |tee mosspectra_${M2}.log
mosback inspecfile=${M2}-fovt.pi elow=${ELO} ehigh=${EHI} ccds="${M2ON}" 2>&1 |tee mosback_${M2}.log

for name in $M1 $M2
do
mkdir ${regname}_${name}
mv ${name}*-${ELO}-${EHI}* ${regname}_${name}
mv ${name}*.pi ${regname}_${name}
mv ${name}*.qdp ${regname}_${name}
mv ${name}*.ps ${regname}_${name}
mv ${name}*imt* ${regname}_${name}
mv ${name}*.arf ${regname}_${name}
mv ${name}*.rmf ${regname}_${name}
mv ${name}*imspdet* ${regname}_${name}

done
'''
)
f.close()

/Users/eusracenorth/Documents/work/XGAP-ABUN/data/RGH80/reduction


In [ ]:
mosspectra eventfile=${M2}-allevc.fits keepinterfiles=yes withregion=yes regionfile=${regname}-${M2}.reg pattern=12 withsrcrem=yes maskdet=${M2}-${regname}regtdet.fits masksky=${M2}-${regname}regtsky.fits elow=${ELO} ehigh=${EHI} ccds="${M2ON}" -V=7 2>&1 |tee mosspectra_${M2}.log
mosback inspecfile=${M2}-fovt.pi elow=${ELO} ehigh=${EHI} ccds="${M2ON}" 2>&1 |tee mosback_${M2}.log

14 pnspectra, pnback

In [100]:
f = open('pnspectra.sh', 'w')

f.write('''
export regname=bkg
export PN=pnS003
export ELO=700
export EHI=1200

for regname in bkg 
do

for pat in 0 4
do
pnspectra eventfile=${PN}-allevc.fits ootevtfile=${PN}-allevcoot.fits keepinterfiles=yes withregion=yes regionfile=${regname}-${PN}.reg pattern=${pat} withsrcrem=yes maskdet=${PN}-bkgregtdet.fits masksky=${PN}-bkgregtsky.fits elow=${ELO} ehigh=${EHI} quads="${PNON}" -V=7 2>&1 |tee pnspectra_${pat}_${PN}.log
pnback inspecfile=${PN}-fovt.pi inspecoot=${PN}-fovtoot.pi elow=${ELO} ehigh=${EHI} quads="${PNON}" 2>&1 |tee pnback_${pat}_${PN}.log

mkdir ${regname}_${PN}_${pat}
mv pn*-${ELO}-${EHI}* ${regname}_${PN}_${pat}
mv pn*.pi ${regname}_${PN}_${pat}
mv pn*.qdp ${regname}_${PN}_${pat}
mv pn*.ps ${regname}_${PN}_${pat}
mv pn*imt* ${regname}_${PN}_${pat}
mv pn*.arf ${regname}_${PN}_${pat}
mv pn*.rmf ${regname}_${PN}_${pat}
mv pn*imspdet* ${regname}_${PN}_${pat}

done
done
'''
)
f.close()

In [ ]:
pnspectra eventfile=${PN}-allevc.fits ootevtfile=${PN}-allevcoot.fits keepinterfiles=yes withregion=yes regionfile=${regname}-${PN}.reg pattern=4 withsrcrem=yes maskdet=${PN}-bkgregtdet.fits masksky=${PN}-bkgregtsky.fits elow=${ELO} ehigh=${EHI} quads="${PNON}" -V=7 2>&1 |tee pnspectra_4_${PN}.log
pnback inspecfile=${PN}-fovt.pi inspecoot=${PN}-fovtoot.pi elow=${ELO} ehigh=${EHI} quads="${PNON}" 2>&1 |tee pnback_4_${PN}.log